In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import random

# Load your CSV data using pandas
data = pd.read_csv('classData.csv')

# Combine the values of the first 4 columns into a new column
data['Combination'] = data.apply(lambda row: ''.join(map(str, row[:4])), axis=1)

# Find unique combinations and assign label numbers
unique_combinations = data['Combination'].unique()
label_mapping = {combination: label for label, combination in enumerate(unique_combinations)}

# Add a new column with the label numbers
data['Label'] = data['Combination'].map(label_mapping)

X = data.iloc[:, 4:10].values  # Features
y = data['Label'].values  # Labels

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a simple neural network model
def create_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(6,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(len(unique_combinations), activation='softmax'))
    return model

# Define the fitness function
def fitness_function(model, X, y):
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, verbose=0)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, np.argmax(y_pred, axis=1))
    return accuracy

# Initialize a population of neural network models with random weights
population_size = 10
population = [create_model() for _ in range(population_size)]

# Evolve the population using a genetic algorithm
num_generations = 10
for generation in range(num_generations):
    # Evaluate the fitness of each model in the population
    fitness_scores = [fitness_function(model, X_train, y_train) for model in population]
    
    # Select the top-performing models (elitism)
    num_parents = 3
    parents = np.argsort(fitness_scores)[-num_parents:]
    
    # Crossover (create new models from the selected parents)
    new_population = [create_model() for _ in range(population_size)]
    for i in range(num_parents, population_size):
        parent1 = random.choice(parents)
        parent2 = random.choice(parents)
        child_weights = []

        # Iterate through the layers and set the weights individually
        for layer in range(len(population[parent1].get_weights())):
            # Perform crossover by taking the mean of the corresponding weights
            layer_weights = np.mean([population[parent1].get_weights()[layer], population[parent2].get_weights()[layer]], axis=0)
            child_weights.append(layer_weights)

        # Set the weights of the child model
        new_population[i].set_weights(child_weights)

    population = new_population


# Find the best model from the final generation
best_model = population[np.argmax(fitness_scores)]
best_accuracy = fitness_scores[np.argmax(fitness_scores)]
print(f"Best accuracy: {best_accuracy}")


50/50 [==============================] - 0s 2ms/step
Best accuracy: 0.8715829624920534


In [5]:
print(best_model.summary())

Model: "sequential_127"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_381 (Dense)           (None, 64)                448       
                                                                 
 dense_382 (Dense)           (None, 32)                2080      
                                                                 
 dense_383 (Dense)           (None, 6)                 198       
                                                                 
Total params: 2726 (10.65 KB)
Trainable params: 2726 (10.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
